In [ ]:
import requests, bs4
from lxml import etree
import pandas as pd
import re
from tqdm import trange
import json

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [ ]:
def adult_crawler(n):
    url = f'https://pornhub.com/video?page={n}'
    response = requests.get(url)
    content = response.content.decode()
    html = etree.HTML(content)

    video_title_o = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//span[@class='title']/a[@title]/text()")
    video_thumb = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//img[@data-thumb_url]/@data-thumb_url")
    video_views = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//span[@class='views']/var/text()")
    video_likes = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//div[@class='value']/text()")
    video_users = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//div[@class='usernameWrap']/a/text()")
    video_url_o = html.xpath("//li[@class='pcVideoListItem js-pop videoblock videoBox']//span[@class='title']/a[@href]/@href")

    video_url=[]
    for i in video_url_o:
        video_url_i = f'https://pornhub.com{i}'
        video_url.append(video_url_i)

    video_title = [re.sub('\n', '', video_title_o[i]) for i in range(len(video_title_o))]
    video_title = [re.sub(' ', '', video_title[i]) for i in range(len(video_title))]

    up_time_o=[]
    #video_time=[]
    video_star_o=[]
    for i in video_url:
        sub_response = requests.get(i)
        sub_content = sub_response.content.decode()
        html = etree.HTML(sub_content)
        video_star_o.append(html.xpath("//a[@class='pstar-list-btn js-mxp']/text()"))
        up_time_o.append(html.xpath("//div[@class='videoInfo']/text()"))
        #video_time.append(html.xpath("//span[@class='mgp_total']/text()"))

    for j in range(len(video_star_o)):
        video_star_o[j] = [re.sub('\n', '', video_star_o[j][i]) for i in range(len(video_star_o[j]))]
        video_star_o[j] = [re.sub('\t', '', video_star_o[j][i]) for i in range(len(video_star_o[j]))]

    video_star_c=[]
    for i in range(len(video_star_o)):
        video_star_c.append(sorted(video_star_o[i]))
    for i in range(len(video_star_c)):
        while '' in video_star_c[i]:
                video_star_c[i].remove('')

    v1=[]
    v2=[]
    for i in range(len(video_star_c)):
            try:
                v1.append(video_star_c[i][0])
            except:
                v1.append('')
            try:
                v2.append(video_star_c[i][1])
            except:
                v2.append('')

    up_time=[]
    for i in range(len(up_time_o)):
        up_time.append(up_time_o[i][0])
        

    video_table = pd.DataFrame({'影片標題':video_title,
                                '觀看次數':video_views,
                                '影片明星1':v1,'影片明星2':v2,
                                #'影片時間':video_time,
                                '上傳時間':up_time,
                                '影片評價':video_likes,
                                '發布者':video_users,
                                '影片縮圖':video_thumb,
                                '影片連結':video_url
                                })

    video_table.insert(1, '影片明星', video_table['影片明星1']+''+video_table['影片明星2'])
    del video_table['影片明星1'],video_table['影片明星2']

    return video_table


In [ ]:
df = pd.concat(adult_crawler(i) for i in trange(11)).reset_index(drop=True)
df.head(100)

In [ ]:
df.to_csv("/Users/syunhua/Desktop/000_prase_python/adult_crawler.csv")